In [1]:
!pip install openai==0.28
!pip install numpy
!pip install tiktoken
!pip install gradio==3.50

In [2]:
import openai
import csv
import json
import os
import numpy as np
from collections import defaultdict
import tiktoken
import gradio as gr

In [3]:
# my api key
openai.api_key = ""

In [4]:
# # preparing dataset in the format of GPT
# {
#     "messages": [
#         { "role": "system" , "content": "ashutoshh helper bot with smart communication" },
#         { "role": "user" , "content": "hey can you explain heap memory in java "},
#         { "role": "assistant", "content": "response of above question"}
#     ],
#     "messages": [
#         { "role": "system" , "content": "ashutoshh helper bot with smart communication" },
#         { "role": "user" , "content": "hey can you explain heap memory setting in  java "},
#         { "role": "assistant", "content": "response of above question to setup java heal memory"}
#     ]
# }
#Load CSV data in
csv_file_path = '/content/sample_data/hotel_hospitality_data.csv'
cleaned_data = []

with open(csv_file_path, 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        for cell in row:
            try:
                # Replace square brackets and inner double quotes that are problematic
                cell = cell.replace('["', '').replace('"]', '').replace('\\"', '"')

                # Load each cell as a JSON object
                cell_json = json.loads(cell)

                # Now that the content is clean, append to cleaned_data list
                cleaned_data.append(cell_json)
            except json.JSONDecodeError as e:
                print(f"JSON decode error for cell '{cell}': {e}")

jsonl_file_path = '/content/sample_data/hotel_hospitality_data_formated.jsonl'
# Write cleaned data to a JSONL file
with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
    for item in cleaned_data:
        jsonl_file.write(json.dumps(item) + '\n')

In [5]:
# Next, we specify the data path and open the JSONL file

data_path = '/content/sample_data/hotel_hospitality_data_formated.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# We can inspect the data quickly by checking the number of examples and the first item

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")


Num examples: 18
First example:
{'role': 'system', 'content': "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}
{'role': 'user', 'content': 'Where is the gym located?'}
{'role': 'assistant', 'content': "Our gym is located on the 2nd floor. I'll be happy to show you the way if you need help!"}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 68, 91
mean / median: 76.38888888888889, 74.0
p5 / p95: 71.4, 87.5

#### Distribution of num_assistant_tokens_per_example:
min / max: 16, 39
mean / median: 24.444444444444443, 23.0
p5 / p95: 19.0, 35.5

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [6]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Calculate the estimated cost for fine-tuning
cost_per_100k_tokens = 0.80  # Cost for every 100,000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 100000) * cost_per_100k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing

Dataset has ~1375 tokens that will be charged for during training
By default, you'll train for 5 epochs on this dataset
By default, you'll be charged for ~6875 tokens
Estimated cost for fine-tuning: approximately $0.06


In [7]:
# Function to save the dataset as a JSONL file
def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

# Specify the path where you want to save the JSONL file in your Google Drive
jsonl_file_path = '/content/sample_data/helper-example-json-clean.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

In [8]:
#Upload data for training
training_file_name = '/content/sample_data/helper-example-json-clean.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

#Gives training file id
print("Training file id:", training_file_id)

Training file id: file-f6MZKsulItHIjd51eLRqd70C


In [9]:
#Create Fine-Tuning Job
suffix_name = "chatner-bot"

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
   # n_epochs = 10
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-P7b6WTON4JGaSe5dUu4BzwgE",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1722909952,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-sW9LYXODuBOJUwL7Lo4RNLZu",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-f6MZKsulItHIjd51eLRqd70C",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": "chatner-bot",
  "seed": 1212571061,
  "estimated_finish": null,
  "integrations": []
}


In [13]:
#list events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 45/90: training loss=0.43
Step 46/90: training loss=1.14
Step 47/90: training loss=0.18
Step 48/90: training loss=0.20
Step 49/90: training loss=0.39
Step 50/90: training loss=0.32
Step 51/90: training loss=0.64
Step 52/90: training loss=0.34
Step 53/90: training loss=0.19
Step 54/90: training loss=0.50
Step 55/90: training loss=0.31
Step 56/90: training loss=0.07
Step 57/90: training loss=0.25
Step 58/90: training loss=0.20
Step 59/90: training loss=0.13
Step 60/90: training loss=0.10
Step 61/90: training loss=0.05
Step 62/90: training loss=0.09
Step 63/90: training loss=0.09
Step 64/90: training loss=0.52
Step 65/90: training loss=0.29
Step 66/90: training loss=0.33
Step 67/90: training loss=0.51
Step 68/90: training loss=0.38
Step 69/90: training loss=0.86
Step 70/90: training loss=0.21
Step 71/90: training loss=0.24
Step 72/90: training loss=0.59
Step 73/90: training loss=0.16
Step 74/90: training loss=0.44
Step 75/90: training loss=0.05
Step 76/90: training loss=0.43
Step 77/

In [14]:
#retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-P7b6WTON4JGaSe5dUu4BzwgE",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1722909952,
  "finished_at": 1722910266,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal:chatner-bot:9t3xDM1s",
  "organization_id": "org-sW9LYXODuBOJUwL7Lo4RNLZu",
  "result_files": [
    "file-a4orKBauNpudoFVar5HnUbhb"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-f6MZKsulItHIjd51eLRqd70C",
  "hyperparameters": {
    "n_epochs": 5,
    "batch_size": 1,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 6695,
  "error": {},
  "user_provided_suffix": "chatner-bot",
  "seed": 1212571061,
  "estimated_finish": null,
  "integrations": []
}

Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:chatner-bot:9t3xDM1s


In [15]:
#Test it out!
test_messages = []

system_message = "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."
test_messages.append({"role": "system", "content": system_message})
user_message = "Where should we park"
test_messages.append({"role": "user", "content": user_message})

print(test_messages)

[{'role': 'system', 'content': "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}, {'role': 'user', 'content': 'Where should we park'}]


In [16]:
#OpenAI Chat Completions to get the response
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, #can test it against gpt-3.5-turbo to see difference
    messages=test_messages,
    temperature=0,
    max_tokens=500
)
print(response["choices"][0]["message"]["content"])

Our parking lot is located right next to the hotel. You can park there for free. I'll be happy to provide directions if needed.


In [3]:
#Gradio for a better UI
def generate_completion(user_prompt):
    hidden_context = ""
    messages = [
        {"role": "system", "content": hidden_context},
        {"role": "user", "content": user_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=fine_tuned_model_id,
        messages=messages,
        max_tokens=100,
        temperature=0
    )
    return response['choices'][0]['message']['content'].strip()

iface = gr.Interface(fn=generate_completion,
                     inputs=gr.inputs.Textbox(lines=5, placeholder='Question about the Airbnb?'),
                     outputs='text',
                     title="Chatner The Airbnb Helper",
                     input_labels="Question",
                     output_labels="Response")

iface.launch(share=True)

<ipython-input-3-50344e08f256>:17: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(lines=5, placeholder='Question about the Airbnb?'),
<ipython-input-3-50344e08f256>:17: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=5, placeholder='Question about the Airbnb?'),
<ipython-input-3-50344e08f256>:17: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=5, placeholder='Question about the Airbnb?'),
<ipython-input-3-50344e08f256>:16: GradioUnusedKwargWarning: You have unused kwarg parameters in Interface, please remove them: {'input_labels': 'Question', 'output_labels': 'Response'}
  iface = gr.Interface(fn=generate_completion,


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.50.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://d1a9a205dcdf3399d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
